In [ ]:
from pyincore import IncoreClient, MappingSet, RestorationService
from pyincore.analyses.epfrestoration import EpfRestoration

In [ ]:
client = IncoreClient()

In [ ]:
epf_rest = EpfRestoration(client)
# Load restoration mapping
restorationsvc = RestorationService(client)
mapping_set = MappingSet(restorationsvc.get_mapping("61f302e6e3a03e465500b3eb"))  # new format of mapping
epf_rest.load_remote_input_dataset('epfs', '5d92355bb9219c06ae7e386a')
epf_rest.set_input_dataset('dfr3_mapping_set', mapping_set)
epf_rest.set_parameter("result_name", "epf_restoration.csv")
epf_rest.set_parameter("restoration_key", "Restoration ID Code")
epf_rest.set_parameter("end_time", 100.0)
epf_rest.set_parameter("time_interval", 1.0)
epf_rest.set_parameter("pf_interval", 0.01)

In [ ]:
epf_rest.run_analysis()

In [ ]:
pf_results = epf_rest.get_output_dataset("pf_results").get_dataframe_from_csv()
pf_results.head()

In [ ]:
time_results = epf_rest.get_output_dataset("time_results").get_dataframe_from_csv()
time_results.head()

In [ ]:
from pyincore.analyses.epfrestoration import EpfRestorationUtil

In [ ]:
epf_rest_util = EpfRestorationUtil(epf_rest)

In [ ]:
functionality = epf_rest_util.get_percentage_func(guid="7f426763-81c6-4cac-b98d-c37a5e439b90",
                                                      damage_state="DS_0", time=2.0)
functionality

In [ ]:
time = epf_rest_util.get_restoration_time(guid="7f426763-81c6-4cac-b98d-c37a5e439b90", damage_state="DS_1", pf=0.81)
time